In [1]:
using Plots
using ParticleFilters
using Distributions
using StaticArrays
using LinearAlgebra
using Random
using StatsBase
using Reel
using POMDPs
#using POMDPSimulators
#using POMDPPolicies
#using POMDPModelTools
using GridInterpolations
using DataStructures
using DataFrames
using CSV

In [2]:
include("atan2.jl")
include("obs.jl")
include("polargrid.jl")
rng = MersenneTwister(2); # A very good random number generator, internal to Julia.

In [3]:
# Creation of the POMDP struct.
# I am unclear as to how this struct is useful in this analysis.

# Implement the POMDP, here.

mutable struct targetPOMDP <: POMDP{Vector{Float64}, Tuple{Float64}, Int64}
    p_walk::Float64
    discount::Float64
end

# statespace = [(10*r,θ*15, crs*15, hdg*15, spd) for r in 0:30, θ in 1:12, crs in 1:12, hdg in 1:12, spd in 1:2]

###

targetPOMDP() = targetPOMDP(0.9, 0.9)
POMDPs.actions(::targetPOMDP) = ((30,1),(0,1),(-30,1),(30, 2), (0,2), (-30,2))
POMDPs.actionindex(::targetPOMDP, a::Tuple) = (a[1]/30+1)*2 + a[2] #cleverness returns 1-6
POMDPs.states(::targetPOMDP) = statespace
POMDPs.stateindex(::targetPOMDP, s::NTuple{5,Int64}) = LinearIndices(statespace)[s[1]/10, s[2]/30, s[3]/30, s[4]/30, s[5]]

###

# POMDPs.stateindex(::targetPOMDP, s::NTuple{5,Int64}) = LinearIndices(statespace)[round(Int,s[1]/10)+1, round(Int,s[2]/30)+1,
#    (round(Int,s[3]/30)+1, round(Int,s[4]/30)+1, round(Int,s[5])+1]

###

POMDPs.observations(::targetPOMDP) = (0, 1, 2, 3)
POMDPs.obsindex(::targetPOMDP, o::Int64) = o + 1

POMDPs.initialstate_distribution(::targetPOMDP) = ParticleCollection([[5, 60, 90, 90, 1] for i in 1:N])
POMDPs.initialstate(::targetPOMDP, rng::AbstractRNG) = [5, 60, 90, 90, 1]
POMDPs.isterminal(::targetPOMDP,s) = s[1] >= 300
POMDPs.discount(::targetPOMDP) = 0.9

###

# General creation of a POMDP.

function POMDPs.gen(m::targetPOMDP, s, a, rng)
    return (sp=f(s,a,rng), r=r(s), o=h(s,rng))
end

In [4]:
# Creation of a POMDP using the initialized struct.

pomdp = targetPOMDP(0.9, 0.9)

targetPOMDP(0.9, 0.9)

In [5]:
# Implementing the next course of action.

#input is course in degrees and rng
#returns next course in degrees

function next_crs(crs,rng)
    if rand(rng) < pomdp.p_walk
        return crs
    end
    crs = (crs + rand(rng,[-1,1])*30) % 360
    if crs < 0 crs += 360 end
    return crs
end

next_crs (generic function with 1 method)

In [6]:
# Implementing the function used for the random walk...
# which tries to understand the the observations created when walking in random directions.

# state as tuple (x, y, crs, hdg) of target (hdg of o/s)

function f(state, control, rng)
    r, θ, crs, hdg, spd = state
    x = r*cos(π/180*θ)
    y = r*sin(π/180*θ)
    pos = [x + TGT_SPD*cos(π/180*crs) - spd*cos(π/180*hdg), y + TGT_SPD*sin(π/180*crs) - spd*sin(π/180*hdg)]
    crs = next_crs(crs,rng)
    hdg = hdg + control[1]
    hdg = hdg % 360
    
    if hdg < 0
        hdg += 360
    end
    
    spd = control[2]
    r = sqrt(pos[1]^2 + pos[2]^2)
    θ = atan2(pos[1],pos[2])*180/π
    
    #if trunc(Int, atan2(pos[1],pos[2])*180/π) == 0
    #    @show pos
    #end
    
    if θ < 0 θ += 360 end
    
    return (r, θ, crs, hdg, spd)::NTuple{5, Real}
end

f (generic function with 1 method)

In [7]:
# Implementing the reward function.

function r(s::NTuple{5,Real})
    range = s[1]
    if range > 150 return -1 end  # reward to not lose track of contact
    if range <= 10 return -1000 end  # collision avoidance
    return 2  # being in "sweet spot" maximizes reward
end

r (generic function with 1 method)

In [8]:
# Implementing the random walk.

POS_0 = [6.0, 60.0]
CRS_0 = 90 # target's course (TARGET)
HDG_0 = 90 # o/s heading (UAV)
SPD_0 = 1 # 1 or 2
TGT_SPD = 1

x = []
courses = []
observs = []

rew = 0

for i in 1:10000
    state = f((POS_0[1], POS_0[2], CRS_0, HDG_0, SPD_0),(0,1),rng)
    POS_0[1], POS_0[2], CRS_0, HDG_0, SPD_0 = state
    θ = state[2]
    rad = state[1]
    if θ < 0 θ += 360 end
    push!(courses, CRS_0)
    push!(observs,(rad, θ, obs0(state), obs1(state), obs2(state), obs3(state)))
    # println(observs[i])
    
    rew += r(state)
    
end

# plot(observs[:,2], observs[:,1], proj=:polar, m=2)

In [9]:
observs_filter = zeros(10000,6)

for iter1 in (1:10000)
    
    for iter2 in (1:6)
        
        observs_filter[iter1, iter2] = observs[iter1][iter2]
        
    end
    
end

In [10]:
# Figure out how to plot this, when you have time for this.

#=

plot(observs_filter[:,2], observs_filter[:,1], proj=:polar, m=2)

=#

In [11]:
##########

# I need to do something with this.

angles = [0, 30, 60, 90, 120, 150, 210, 240, 270, 300, 330]

statespace = thestates
actionspace = ((30,1), (0,1), (-30,1), (30, 2), (0,2), (-30,2))

action_index(a) = trunc(Int, 2*(a[1]/30+1) + a[2])
actions_ = ((-30,1), (-30, 2), (0, 1), (0, 2), (30, 1), (30, 2))

observations = (0, 1, 2, 3)

##########

(0, 1, 2, 3)

In [12]:
### SIR Particle Filter - I don't know what this is used for.

N = 500
updater = SIRParticleFilter(pomdp, N);

### This changes the original "random walk".
### In this case, this will be used for the implementation of the reinforcement learning algorithm.

function f2(x, u, rng)
    temp = [i for i in f(x, u, rng)]
    return temp
end

f2 (generic function with 1 method)

In [13]:
# This initializes (some grid interpolation ... ???) the rewards for all combinations of states.

#θ = zeros(length(grid),6);
θ = [r(Tuple(ind2x(grid, j))) for j in 1:length(grid), i in 1:6];

In [14]:
N = 500
model = ParticleFilterModel{Vector{Float64}}(f2, g)
pfilter = SIRParticleFilter(model, N);
α = 0.5
γ = 0.95
ϵ = 0.3
x = [20, 60, 90, 90, 1];
λ = 0.8
b = ParticleCollection([[20, 60, 90, 90, 1] for i in 1:N]);
counter = 0;

In [15]:
#=

## Q-learning loop
plots = [];
betas = Deque{Array}();
β = zeros(length(grid),6);

epochs = 1000;
last = 0;

total = 0;
ξ = weighted_grid_2(b)/N;

=#

In [16]:
#=

counter = 0;
i = 1;
counter += 1;
u = next_action([transpose(θ[:,j])*ξ for j in 1:size(θ)[2]], ϵ, rng);

#observe new state and reward
xp = f2(x, actions_[u], rng);
o = h(xp, rng); 
b = update(pfilter, b, actions_[u], o); # filter, belief states, action, observation

rew = r(Tuple(xp));
ξ = weighted_grid_2(b)/N;
β[:,u] = ξ;

total += rew
#v = 10^3*sqrt(var(ξ))
if length(betas) < 20
    pushfirst!(betas, β)
else
    pop!(betas)
    pushfirst!(betas, β)
end

=#

In [17]:
##########

In [18]:
# fast-informed bound (FIB) loop

### Need to do, later.

In [19]:
actions = ((-30,1), (-30, 2), (0, 1), (0, 2), (30, 1), (30, 2));
observations = (0, 1, 2, 3);
# θ = [r(Tuple(ind2x(grid, j))) for j in 1:length(grid), i in 1:6];

In [20]:
# Individual States for each state parameter.

states_r = [10.0*r for r in 1:30]
states_θ = [30.0*θ for θ in 0:12]
# states_crs = [30*c for c in 1:12]
states_hdg = [30*h for h in 1:12]
states_spd = [1,2];

In [21]:
# Creating a matrix for all possible state combinations.

length_states = length(states_r) * length(states_θ) * length(states_hdg) * length(states_spd) # length(states_crs) * 
# states = zeros(length_states, 5);
states = RectangleGrid(states_r, states_θ, states_hdg, states_spd) # , states_crs

RectangleGrid{4}([10.0, 20.0, 30.0, 40.0, 50.0, 60.0, 70.0, 80.0, 90.0, 100.0  …  210.0, 220.0, 230.0, 240.0, 250.0, 260.0, 270.0, 280.0, 290.0, 300.0],[0.0, 30.0, 60.0, 90.0, 120.0, 150.0, 180.0, 210.0, 240.0, 270.0, 300.0, 330.0, 360.0],[30.0, 60.0, 90.0, 120.0, 150.0, 180.0, 210.0, 240.0, 270.0, 300.0, 330.0, 360.0],[1.0, 2.0],)

In [22]:
# Looping through all possible state combinations and inserting them into the "states" matrix.

#=

count_row = 0

for r in states_r
    
    for θ in states_θ
        
        for crs in states_crs
            
            for hdg in states_hdg
                
                for spd in states_spd
                    
                    count_row += 1
                    
                    states[count_row, 1] = r
                    states[count_row, 2] = θ
                    states[count_row, 3] = crs
                    states[count_row, 4] = hdg
                    states[count_row, 5] = spd
                    
                end
                
            end
            
        end
        
    end
    
end

=#

In [23]:
# Particle Filter Parameters

N = 500;
model = ParticleFilterModel{Vector{Float64}}(f2, g)
pfilter = SIRParticleFilter(model, N);
b_particle = ParticleCollection([[20, 60, 90, 90, 1] for i in 1:N]);

In [24]:
# Update Belief

function UpdateBelief(beta, act, obs)
   
    beta_new = zeros(length(beta), 1);
    
    P = 0;
    sts_count = 0;
    
    for sts in states
        
        sts_prime_count = 0;
        obs_trans_sum = 0;
        
        for sts_prime in states
           
            sts_prime_count += 1;
            obs_trans_sum += g(nothing, act, sts_prime, obs) * 0.9;
            
        end
        
        sts_count += 1;
        P += beta[sts_count] * obs_trans_sum;
        
    end
    
    sts_beta_new_count = 0;
    
    for sts_beta_new in states
       
        trans_belief_sum = 0;
        sts_belief_count = 0;
        
        for sts_belief in states
            
            sts_belief_count += 1;
            trans_belief_sum += 0.9 * beta[sts_belief_count];
            
        end
        
        sts_beta_new_count += 1;
        beta_new[sts_beta_new_count] = ( g(nothing, act, sts_beta_new, obs) / P ) * trans_belief_sum
        
    end
    
    println(length(beta_new))
    
    return beta_new
    
end

UpdateBelief (generic function with 1 method)

In [25]:
# Backup Belief Update

function BackupBelief(lambda, beta)
    
    alpha_all = zeros(size(lambda)[1], 6)
    
    count_a = 0
    
    for act in actions
        
        count_a += 1
        
        alpha_o = zeros(size(lambda)[1], 4)
        
        count_o = 0
        
        for obs in observations
            
            betaP = beta # UpdateBelief(beta, act, obs)
            
            count_o += 1
            arg_max_alpha_o = maximum(LinearIndices(CartesianIndices(findmax( transpose(lambda) * betaP )[2])))
            alpha_o[:,count_o] += lambda[:,arg_max_alpha_o]
            
        end
        
        count_s = 0
        
        for sts in states
            
            count_o = 0
            
            count_s += 1
            
            obs_alpha_act_obs = 0
            
            for obs_sum in observations
                
                count_o += 1
            
                obs_alpha_act_obs += g(nothing, act, sts, obs_sum) * 0.9 * alpha_o[count_s, count_o]
                
            end
            
            alpha_all[count_s, count_a] = r(tuple(sts)) + γ * obs_alpha_act_obs
            
        end
        
    end
        
    arg_max_alpha_all = maximum(LinearIndices(CartesianIndices(findmax( transpose(alpha_all) * beta )[2])))
    alpha_return = alpha_all[:,arg_max_alpha_all]
    
    println("---")
    println(size(alpha_return))
    println("---")
    
    return alpha_return
    
end

BackupBelief (generic function with 1 method)

In [26]:
#=

a = [1 3 54 6 89 0 3]
maximum(LinearIndices(CartesianIndices(findmax(a)[2])))

=#

# reward(tuple(states[1]))

In [81]:
# Randomized Point Based Update

function RandomizedPointBasedUpdate(BETA, Lambda)
    
    LambdaP = [] # Array{Float64,1}
    BETA_P = BETA
    
    while (BETA_P != [])
        
        rand_num = rand(1:length(BETA_P))
        beta = BETA_P[rand_num]
        
        alpha = BackupBelief(beta, Lambda)
        
        cond_left = transpose(alpha) * beta
        cond_right = findmax( transpose(Lambda) * beta )[1]
        
        # println(cond_left >= cond_right)
        
        if (cond_left >= cond_right)
            
            push!(LambdaP, alpha)
            
        else
            
            arg_max_alpha = maximum(LinearIndices(CartesianIndices(findmax( transpose(Lambda) * beta )[2])))
            println(arg_max_alpha)
            alphaP = Lambda[arg_max_alpha]
            push!(LambdaP, alphaP)
            
        end
        
        BETA_P_new = []
        
        count_beta = 0
        
        for beta_P in BETA_P
            
            count_beta += 1
            println(count_beta)
            
            # println(size(LambdaP[1]))
            
            # println(size(LambdaP))
            
            cond_left_new = findmax( transpose(LambdaP[1]) * beta_P )[1]
            cond_right_new = findmax( transpose(Lambda) * beta_P )[1]
            
            # println(findmax( transpose(LambdaP) * beta_P ))
            # println(findmax( transpose(Lambda) * beta_P ))
            
            println(cond_left_new < cond_right_new)
            
            if ( cond_left_new < cond_right_new )
            
                push!(BETA_P_new, beta_P)
                    
            end
            
        end
        
        BETA_P = BETA_P_new
        
    end
    
    return LambdaP
    
end

RandomizedPointBasedUpdate (generic function with 1 method)

In [82]:
# Creating the value to initialize alpha with.

state_min_rew = [1, 0, 0, 0, 0];
min_rew = r(Tuple(state_min_rew));
alpha_init = 1 / (1 - γ) * min_rew;

In [83]:
# Initializing the alpha array.

alpha_start = zeros(length_states, 6);
alpha_start .+= alpha_init;

In [84]:
# Initializing the beta belief state.

beta_start = zeros(length_states, 6);
beta_start .+= (1/length_states);

beta_start_tuple = []

for counter in 1:6
    
    push!(beta_start_tuple, beta_start[:,counter])
    
end

In [85]:
# beta_start_tuple;

In [86]:
test_rand = RandomizedPointBasedUpdate(beta_start_tuple, alpha_start)

---
(9360,)
---
1
false
2
false
3
false
4
false
5
false
6
false


1-element Array{Any,1}:
 [-999.9999086538462, 2.0000913461538463, 2.0000913461538463, 2.0000913461538463, 2.0000913461538463, 2.0000913461538463, 2.0000913461538463, 2.0000913461538463, 2.0000913461538463, 2.0000913461538463  …  -0.9999086538461538, -0.9999086538461538, -0.9999086538461538, -0.9999086538461538, -0.9999086538461538, -0.9999086538461538, -0.9999086538461538, -0.9999086538461538, -0.9999086538461538, -0.9999086538461538]

In [31]:
# Implementing the reward function.

function r(s::Tuple{Array{Float64,1}})
    range = s[1][1]
    if range > 150 return -1 end  # reward to not lose track of contact
    if range <= 10 return -1000 end  # collision avoidance
    return 2  # being in "sweet spot" maximizes reward
end

r (generic function with 2 methods)

In [ ]:
##########

In [ ]:
### Initializing the alpha vestors for all possible states

alphaQMDP = zeros(length_states, 6);
# findmax(alphaQMDP[1,:])[1]

In [ ]:
### Fully Observable Value Approximation - QMDP

function QMDP(alpha_start)
    
    for k in 1
    
        state_count = 0
    
        for sts in states
        
            state_count += 1
            action_count = 0
            println(state_count)
        
            for act in actions
            
                action_count += 1;
                transition_action_sum = 0;
                state_prime_count = 0
            
                for sts_prime in states
                
                    state_prime_count += 1;
                    transition_action_sum += γ * 0.9 * findmax(alpha_start[state_prime_count,action_count])[1]
                
                end
            
                alpha_start[state_count, action_count] = r(tuple(sts)) + transition_action_sum
            
            end       
        
        end
        
    end
    
    return alpha_start
    
end

In [ ]:
### Fully Observable Value Approximation - FIB

function FIB(alpha_start)
    
    for k in 1
    
        state_count = 0;
    
        for sts in states
        
            state_count += 1;
            action_count = 0;
            println(state_count)
        
            for act in actions
            
                action_count += 1;
                obs_count = 0;
                sum_obs = 0;
            
                for obs in observations
                
                    act_prime_count = 0;
                    sum_state_action = zeros(1, 6);
                    
                    for act_prime in actions
                        
                        act_prime_count += 1;
                        sts_prime_count = 0;
                        
                        for sts_prime in states
                            
                            sts_prime_count += 1
                            sum_state_action[1,act_prime_count] += g(nothing, act_prime, sts_prime, obs) * 0.9 * alpha_start[sts_prime_count, act_prime_count];
                            
                        end
                        
                    end
                    
                    sum_obs += findmax(sum_state_action)[1];
                    
                end
                
                alpha_start[state_count, action_count] += r(tuple(sts)) + γ * sum_obs;
                
            end       
        
        end
        
    end
    
    return alpha_start
    
end

In [ ]:
# Backup Belief for the Point-Based Value Iteration

### NEED TO FINISH!!!

function BackupBelief(Lambda, beta)
    
    for act in actions
        
        for obs in observations
            
            b_prime = UpdateBelief(beta, act, obs)
            
            alpha[act_count, obs_count] = findmax( transpose(Lambda) * beta_prime )
            
        end
        
    end
    
    for sts in states
        
        alpha = 
        
    end
    
    alpha = findmax( transpose(alpha) * beta )
    
end

In [ ]:
test_QMDP = QMDP(alphaQMDP);

In [ ]:
test_FIB = FIB(alphaQMDP);

In [ ]:
for counter in 1:9360
    
    println(test_FIB[counter,1] - test_FIB[counter,2])
    
end

In [ ]:
##########

In [ ]:
## Q-learning loop
plots = []
betas = Deque{Array}()
β = zeros(length(grid),6);

epochs = 1000
last = 0

total = 0
ξ = weighted_grid_2(b)/N
for i in 1:(1500*epochs)
    counter += 1
    
    # choose next action
    u = next_action([transpose(θ[:,j])*ξ for j in 1:size(θ)[2]], ϵ, rng)
    
    #observe new state and reward
    xp = f2(x, actions_[u], rng)
    obs = h(xp, rng)
    b = update(pfilter, b, actions_[u], obs)
    rew = r(Tuple(xp))
    
    ξ = weighted_grid_2(b)/N # This is the interpolation.
    
    β[:,u] = ξ
    
    total += rew
    #v = 10^3*sqrt(var(ξ))
    if length(betas) < 20
        pushfirst!(betas, β)
    else
        pop!(betas)
        pushfirst!(betas, β)
    end
        
    cur = (rew + γ * max2([transpose(θ[:,j])*ξ for j in 1:size(θ)[2]], rng) - last)
    #println(cur)

    #update θ
    #θ += α * cur *β
    for (j, bet) in enumerate(betas)
        θ += (λ^j) * α * cur * bet
    end
    
    last = transpose(θ[:,u])*ξ
    
    x = xp
    
    #=
    
    if counter % 1500 == 0
        push!(totals, total/2)
        #ϵ = max(min((20000 - 2*total)/180000, 1), 0)
        println("--------- CURRENT: ", total/2, " AVG: ", mean(totals), " Epoch: ", 
            trunc(Int, counter/500), " -----------")
        total = 0
        xp = [rand(rng, 25:120), rand(rng,0:360), rand(rng,0:11)*30, 1, 1];
        b = ParticleCollection([xp[1:4] for i in 1:N]);
        last = 0
        sleep(3)
    end
    
    =#
    
    #plotting
    #r_ = [row[1] for row in particles(b)]
    #theta = [row[2] for row in particles(b)]*π/180
    #x_theta = x[2]*π/180
    #x_r = x[1]
    
    #print(".")
    #plt = plot(proj=:polar, lims=(0,200), size=(1000,1000))
    #scatter!(theta, r_, markersize=1, label="particles")
    #scatter!([x_theta], [x_r], markersize=3, label="target")
          
    #push!(plots, plt)
    
end

In [ ]:
env = POMDPEnvironment(targetPOMDP())

function simulate(env::AbstractEnvironment, nsteps::Int = 100)
    done = false
    r_tot = 0.0
    step = 1
    o = reset!(env)
    while !done && step <= nsteps
        action = sample_action(env) # take random action 
        obs, rew, done, info = step!(env, action)
        obs = trunc(Int,obs[1])
        #@show obs, rew, done, info
        r_tot += rew
        step += 1
    end
    return r_tot
end

@show simulate(env)
#ac = sample_action(env)
#step!(env, ac)

# Link: https://github.com/JuliaPOMDP/RLInterface.jl